**Projede kullanılacak kütüphanelerin import edilmesi:**

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
import matplotlib.pyplot as plt
import seaborn as sns

**Veri seti ve veri görünürlük ayalarları:**

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

**Veri setinin import edilmesi:**

In [3]:
df_ = pd.read_csv("crm_analytics/flo_data_20k.csv")
df = df_.copy()

**Veri setinin rastgele 5 gözlemi:**

In [4]:
np.random.seed(115)
df.sample(5)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
10788,f33f4646-1a94-11ea-98d5-000d3a38a36f,Ios App,Ios App,2019-11-21,2020-11-08,2020-11-08,2019-11-21,3.00,1.00,89.99,377.45,"[AKTIFCOCUK, COCUK]"
3977,47d3b212-7361-11ea-92d0-000d3a38a36f,Mobile,Mobile,2019-12-08,2021-02-26,2021-02-26,2020-02-09,1.00,2.00,169.98,234.97,"[KADIN, AKTIFSPOR]"
8133,bf042a06-54e3-11eb-9e65-000d3a38a36f,Ios App,Offline,2021-01-12,2021-04-12,2021-01-12,2021-04-12,1.00,1.00,129.98,95.99,[AKTIFSPOR]
3900,8cb2eece-f8a1-11e9-b138-000d3a38a36f,Mobile,Mobile,2019-10-27,2021-03-19,2021-03-19,2019-11-26,1.00,2.00,529.95,105.15,[]
3458,5e718ac8-b109-11e9-9757-000d3a38a36f,Mobile,Offline,2019-05-09,2020-07-03,2019-05-09,2020-07-03,1.00,2.00,149.98,109.99,"[KADIN, AKTIFSPOR]"


**Veri setine genel bakış ve değişken analizi:**

(**NOT:** Veri setinine genel bakış ve değişken analizi adımında veri setinde yer almayan toplam sipariş sayısı ve toplam ödenen ücret verileri veri setine _"order_num_total"_ ve _"customer_value_total"_ değişken isimleriyle eklenmiştir.)

In [5]:
def overview_and_analysis(dataframe):
    
    # Online ve offline olarak yapılmış olan alışveriş adetlerinin toplam sayısı:
    dataframe["order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    # Online ve offline olarak yapılmış olan alışverişlerin toplam tutarı:
    dataframe["customer_value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    
    # Tarih değişkenleri:
    date_cols = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_cols] = dataframe[date_cols].apply(pd.to_datetime)
    # Kategorik değişkenler:
    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ["category", "object", "bool"]]
    # Sayısal görünüp kategorik olan değişkenler:
    num_but_cat = [col for col in dataframe.columns if
                   dataframe[col].nunique() < 10 and dataframe[col].dtypes in ["int64", "int32", "float64", "float32"]]
    # Kategorik görünüp kardinal olan değişkenler:
    cat_but_car = [col for col in dataframe.columns if
                   dataframe[col].nunique() > 20 and str(dataframe[col].dtypes) in ["category", "object"]]
    # Kardinal değişkenleri kategorik değişkenlerden çıkarma:
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    # Sayısal değişkenler
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes in ["int64", "int32", "float64", "float32"]]
    # Sayısal görünümlü kategorik değişkenleri sayısal değişkenlerden çıkarma:
    num_cols = [col for col in num_cols if col not in cat_cols]

    print("Veri Setine Genel Bakış:", "-------------------------", sep="\n")
    print("Gözlem/Değişken Sayısı:", "-------------------------", dataframe.shape, "-------------------------",sep="\n")
    print("Boş Gözlem Sayısı:", "-------------------------",[col + "-> " + str(dataframe[col].isnull().sum()) for col in dataframe.columns], "-------------------------",sep="\n")
    print("Değişken Tipleri:", "-------------------------",[col + "-> " + str(dataframe[col].dtype) for col in dataframe.columns], "-------------------------", sep="\n")
    print("Toplam değişken sayısı:", "-------------------------", len(df.columns), "-------------------------",sep="\n")
    print("Sayısal Değişkenler:", "-------------------------", [col for col in num_cols], "-------------------------",sep="\n")
    print("Sayısal Değişken Sayısı: ", len(num_cols), "\n", "-------------------------")
    print("Kategorik Değişkenler:", "-------------------------", [col for col in cat_cols], "-------------------------",sep="\n")
    print("Kategorik Değişken Sayısı: ", len(cat_cols), "\n", "-------------------------")
    print("Kardinal Değişkenler:", "-------------------------", [col for col in cat_but_car],"-------------------------", sep="\n")
    print("Kardinal Değişken Sayısı: ", len(cat_but_car), "\n", "-------------------------")
    print("Tarihsel Değişkenler:", "-------------------------", [col for col in date_cols], "-------------------------",sep="\n")
    print("Tarihsel Değişken Sayısı: ", len(date_cols), "\n", "-------------------------")
    return df,num_cols,cat_cols,cat_but_car,date_cols

df,num_cols,cat_cols,cat_but_car,date_cols = overview_and_analysis(df)

Veri Setine Genel Bakış:
-------------------------
Gözlem/Değişken Sayısı:
-------------------------
(19945, 14)
-------------------------
Boş Gözlem Sayısı:
-------------------------
['master_id-> 0', 'order_channel-> 0', 'last_order_channel-> 0', 'first_order_date-> 0', 'last_order_date-> 0', 'last_order_date_online-> 0', 'last_order_date_offline-> 0', 'order_num_total_ever_online-> 0', 'order_num_total_ever_offline-> 0', 'customer_value_total_ever_offline-> 0', 'customer_value_total_ever_online-> 0', 'interested_in_categories_12-> 0', 'order_num_total-> 0', 'customer_value_total-> 0']
-------------------------
Değişken Tipleri:
-------------------------
['master_id-> object', 'order_channel-> object', 'last_order_channel-> object', 'first_order_date-> datetime64[ns]', 'last_order_date-> datetime64[ns]', 'last_order_date_online-> datetime64[ns]', 'last_order_date_offline-> datetime64[ns]', 'order_num_total_ever_online-> float64', 'order_num_total_ever_offline-> float64', 'customer_va

**SAYISAL DEĞİŞKENLERİN ANALİZİ**

**Sayısal değişkenlerin betimsel istatistikleri:**

In [6]:
df[num_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.00,3.11,4.23,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.00,1.91,2.06,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.00,253.92,301.53,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.00,497.32,832.60,12.99,149.98,286.46,578.44,45220.13
order_num_total,19945.00,5.02,4.74,2.00,3.00,4.00,6.00,202.00
customer_value_total,19945.00,751.24,895.40,44.98,339.98,545.27,897.78,45905.10


Sayısal değişkenlerin betimsel istatistikleri incelendiğinde veri seti dağılımında aykırılıklar olduğu görülmektedir. Bu sebepten dolayı aykırı değerleri baskılayarak veriyi işlemek gerekir. Baskılama işleminin yapılma sebebi satış verilerinde yer alan aykırılıkların gidirilerek tahminle işleminin daha yansız bir şekilde gerçekleştirilmek istenmesidir.

**Aykırı değerlerin eşik değerlerine baskılanması:**

In [7]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit,0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit,0)

for i in num_cols:
    replace_with_thresholds(df,i)

In [8]:
df[num_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.00,3.09,3.81,1.00,1.00,2.00,4.00,48.00
order_num_total_ever_offline,19945.00,1.89,1.43,1.00,1.00,1.00,2.00,16.00
customer_value_total_ever_offline,19945.00,251.92,251.02,10.00,99.99,179.98,319.97,3020.00
customer_value_total_ever_online,19945.00,489.71,632.61,12.99,149.98,286.46,578.44,7800.00
order_num_total,19945.00,5.00,4.26,2.00,3.00,4.00,6.00,52.00
customer_value_total,19945.00,743.39,701.94,44.98,339.98,545.27,897.78,8845.00


Veri setininde yer alan sayısal değişkenlerin değişken bazlı olarak 0.01 ve 0.99 çeyrekliklerine göre belirlenmiş eşik değerlerine baskılanması sonucu verilerin betimsel istatistikleri yukarıda yer alan tablodaki gibi şekillenmiştir.

**CLTV HESAPLAMALARI**

In [9]:
def create_cltv_df(dataframe,prediction_month = 6):
    # CLTV veri yapısının oluşturulması:

    # Analiz tarihinin belirlenmesi (Son alışveriş tarihinden 2 gün sonrası olarak belirlenmiştir.)
    dataframe["last_order_date"].max()  # 2021-05-30
    analysis_date = dt.datetime(2021, 6, 1)

    # CLTV analizi için verinin tablolaştırma adımları:

    cltv_df = pd.DataFrame()
    # Kullanıcıların unique id derğerleri:
    cltv_df["master_id"] = dataframe["master_id"]
    # Müşterilerin recency (son alışveriş tarih - ilk alışveriş yaptığı tarih) değerinin haftalık cinsten değerleri:
    cltv_df["recency_cltv_weekly"] = ((dataframe["last_order_date"] - dataframe["first_order_date"]).dt.days) / 7
    ((df["last_order_date"]- df["first_order_date"]).astype('timedelta64[D]')) / 7
    # Müşterilerin T (analiz tarihi - ilk yaptığı alışveriş tarihi) değerinin haftalık cinsten değerleri (Müşterinin şirket yaşı olarakta bilinir.):
    cltv_df["T_weekly"] = ((analysis_date - dataframe["first_order_date"]).dt.days) / 7
    # Müşterilerin alışveriş yapma sıklığı değerleri:
    cltv_df["frequency"] = dataframe["order_num_total"]
    # Müşterilerin toplam alışveriş tutarlarının ortalamaları değerleri:
    cltv_df["monetary_cltv_avg"] = dataframe["customer_value_total"] / dataframe["order_num_total"]
    # Alışveriş yapan kişilerden "müşteri" statüsündeki kişilerin seçilmesi:
    cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

    # Sipariş sayısının tahmini adına BG-NBD modelinin kurulması:
    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['frequency'],
            cltv_df['recency_cltv_weekly'],
            cltv_df['T_weekly'])

    # BG-NBD modelinin kurulup fit edilmesi ardından satışların tahminlenmesi:

    # Müşteri bazlı belirlenen ay süresince aylık sipariş tahminleri:
    cltv_df["exp_sales_"+str(prediction_month)+"_month"] = bgf.predict(4 * prediction_month,
                                                                       cltv_df['frequency'],
                                                                       cltv_df['recency_cltv_weekly'],
                                                                       cltv_df['T_weekly'])

    # Müşterilerin harcamalarının tahmini adına Gamma-Gamma modelinin kurulması:
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
    cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                           cltv_df['monetary_cltv_avg'])

    # Müşterilerin CLTV (Müşterilerin şirkete potansiyel getirileri) değerlerinin hesaplanması:
    cltv = ggf.customer_lifetime_value(bgf,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'],
                                       cltv_df['monetary_cltv_avg'],
                                       time=prediction_month,
                                       freq="W",
                                       discount_rate=0.01)
    cltv_df["cltv"] = cltv

    # Müşterilerin CLTV değerlerine göre segmentlere ayırılması:
    cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["Düşük", "Ortalama", "Yüksek", "Çok Yüksek"])

    return cltv_df

cltv_df = create_cltv_df(df)


**NOT:** Fonksiyonun default halinde tahminleme yapılacak süre 6 aydır. İlgili parametre için tahminleme yapılması istenen ay metriği değiştirilebilir.

In [10]:
cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_6_month,exp_average_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87,1.97,193.63,399.91,Çok Yüksek
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88,1.97,96.66,200.17,Yüksek
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06,1.35,120.97,170.92,Yüksek
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98,1.41,67.32,99.66,Düşük
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99,0.79,114.32,94.76,Düşük


In [11]:
cltv_df.groupby("cltv_segment").agg({"cltv":["min","mean","max"]})

cltv               
                min   mean     max
cltv_segment                      
Düşük         12.01  80.22  112.28
Ortalama     112.28 138.43  165.65
Yüksek       165.66 199.99  240.98
Çok Yüksek   241.00 365.43 5326.21